In [1]:
import os
import sys

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

print(f"This is the repo dir {repo_dir}")

This is the repo dir /Users/kwasia/Documents/Projects/ai-starter-kit


In [2]:
import pandas as pd
from utils.eval.rag_eval import (
    RAGEvaluator,
    RAGEvalConfig,
    load_pipeline,
    load_eval_dataframe,
)
from langchain_community.llms.sambanova import SambaStudio, Sambaverse
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

/Users/kwasia/.pyenv/versions/3.11.3/envs/adi-rag-eval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load config
config_path = "../../utils/eval/config.yaml"
config = RAGEvalConfig(config_yaml_path=config_path)

In [5]:
# Create evaluator
eval_llms = []
for conf in config.eval_llm_configs:
    print("conf:", conf)
    llm_name, llm_config = config.get_llm_config(conf)
    print("llm_name:", llm_name)
    print("llm_config:", llm_config)
    eval_llm = SambaStudio(**llm_config)
    eval_llms.append((llm_name, eval_llm))

print("eval_llms:", eval_llms)

eval_embeddings = HuggingFaceInstructEmbeddings(model_name=config.embedding_model_name)
evaluator = RAGEvaluator(eval_llms, eval_embeddings, config_path)

conf: ('EVAL_LLAMA38B', {'name': 'EVAL_LLAMA38B', 'model_kwargs': {'select_expert': 'Meta-Llama-3-8B-Instruct', 'process_prompt': False, 'max_tokens_to_generate': 512}})
llm_name: EVAL_LLAMA38B
llm_config: {'sambastudio_base_url': 'https://sjc3-svqa.sambanova.net', 'sambastudio_project_id': '0c027dd1-b0d3-4928-ad7f-515c613ad04c', 'sambastudio_endpoint_id': '3ef938b5-b1d6-4814-8086-cfe38e90bf0b', 'sambastudio_api_key': '6db47fcf-bdd0-48d6-b4b1-0bc7d0df823d', 'model_kwargs': {'select_expert': 'Meta-Llama-3-8B-Instruct', 'process_prompt': False, 'max_tokens_to_generate': 512}}
conf: ('EVAL_LLAMA370B', {'name': 'EVAL_LLAMA370B', 'model_kwargs': {'select_expert': 'Meta-Llama-3-70B-Instruct', 'process_prompt': False, 'max_tokens_to_generate': 512}})
llm_name: EVAL_LLAMA370B
llm_config: {'sambastudio_base_url': 'https://sjc3-svqa.sambanova.net', 'sambastudio_project_id': '0c027dd1-b0d3-4928-ad7f-515c613ad04c', 'sambastudio_endpoint_id': '3ef938b5-b1d6-4814-8086-cfe38e90bf0b', 'sambastudio_api

In [ ]:
# Use Case 1: CSV file with pre-generated answers, no context
eval_df = pd.read_csv("../data/res.csv")
results1 = evaluator.evaluate(eval_df)
print("Results 1:", results1)

In [ ]:
# Use Case 2: CSV file with pre-generated answers and context
eval_df = pd.read_csv("eval_set_with_answers_and_context.csv")
results2 = evaluator.evaluate(eval_df)
print("Results 2:", results2)

In [6]:
# Use Case 3: CSV file without answers, generate with pipelines, no context
eval_df = pd.read_csv("../data/res.csv")
pipelines = [
    load_pipeline(SambaStudio(**llm_config), config)
    for _, llm_config in [config.get_llm_config(conf) for conf in config.llm_configs]
]
results3 = evaluator.evaluate(eval_df, pipelines)
print("Results 3:", results3)

load INSTRUCTOR_Transformer
max_seq_length  512


/Users/kwasia/.pyenv/versions/3.11.3/envs/adi-rag-eval/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


AttributeError: 'SambaStudio' object has no attribute 'model_name'

In [ ]:
# Use Case 4: CSV file without answers, generate with pipelines, with context from vector DB
eval_df = pd.read_csv("eval_set.csv")
pipelines = [
    load_pipeline(
        SambaStudio(
            sambastudio_base_url=conf["base_url"],
            sambastudio_project_id=conf["project_id"],
            sambastudio_endpoint_id=conf["endpoint_id"],
            sambastudio_api_key=conf["api_key"],
            **conf["model_kwargs"]
        ),
        config,
    )
    for conf in config.llm_configs
]
results4 = evaluator.evaluate(eval_df, pipelines)
print("Results 4:", results4)

In [ ]:
# Use Case 5: Evaluate on HF dataset
config.config["eval_dataset"]["hf_dataset_name"] = "squad_v2"
eval_df = load_eval_dataframe(config)
pipelines = [
    load_pipeline(
        SambaStudio(
            sambastudio_base_url=conf["base_url"],
            sambastudio_project_id=conf["project_id"],
            sambastudio_endpoint_id=conf["endpoint_id"],
            sambastudio_api_key=conf["api_key"],
            **conf["model_kwargs"]
        ),
        config,
    )
    for conf in config.llm_configs
]
results5 = evaluator.evaluate(eval_df, pipelines)
print("Results 5:", results5)